# Ninety One Technical Assessment – Portfolio Risk Analysis  
### *Prepared by Nicholas de Clercq*

This notebook explores the characteristics, risk dynamics, and ESG profile of an equity portfolio provided for analysis.  
All analytics and visualisations are performed in **Python**, with results designed to demonstrate both technical proficiency and interpretive insight.

---

## Project Overview

The analysis is structured in four parts:

1. **Current Portfolio Characteristics**  
   - Composition by sector, country, and position size  
   - Concentration and diversification metrics  
   - Activeness versus the benchmark  

2. **Return Analysis**  
   - Returns  
   - Active returns
   - Fund size through time
   - Return attribution  
     - Top 10 share contributors and detractors  
     - Sector return attribution  
     - Region return attribution  
   - Sector exposures through time  
   - Regional exposures through time  

3. **Risk Analysis**  
   - Total risk and active risk trends  
   - beta evolution  
   - Contribution to risk by sector, region, and asset 
   - drawdowns, sharpe ratio, rolling standard deviation

4. **ESG Analysis**  
   - Weighted ESG scores and trends  
   - Breakdown of Environmental, Social, and Governance dimensions 

---

### Objective

The goal is to identify and clearly articulate the **key drivers of portfolio risk, performance, and ESG trends** —  
and to demonstrate how these factors have evolved over time.

---

*All charts are interactive and were generated using `plotly.express` with a custom Ninety One colour palette.*


In [155]:
import importlib, portfolio_utils as pu
importlib.reload(pu)
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
import plotly.io as pio
pio.renderers.default = "notebook_connected"

# Optional: sizing/style
px.defaults.width = 580
px.defaults.height = 420
px.defaults.template = "plotly_white"
# jupyter nbconvert --to html --no-input "portfolio_analysis_jupyter.ipynb"
# jupyter nbconvert --execute --to html --no-input "portfolio_analysis_jupyter.ipynb"


from portfolio_utils import *

### 1. Current Portfolio Characteristics

In the following section we can take a look at the portfolio's current characteristics. Starting with the current holdings of the portfolios.

In [156]:
df = load_data("Risk Analyst Case study_092025.csv")
df = add_derived_columns(df)
pa = PortfolioAnalyzer(df)


In [157]:
top10_pie_latest(df, by="Weight (%)").show()

In [158]:
sector_pie_latest(df).show()

In [159]:
country_pie_latest(df).show()

This portfolio shows a large exposure to emerging market countires such as China and India as well as exposures to South American countires and South Africa, typical of a fund that would be benchmarked to the the MSCI emerging markets index.

To Further understand the current drivers of risk in the portfolio we can take a look at the most recent contributors to the overall risk of the portfolio.

In [160]:
risk_contrib_bar(df).show()

In [161]:
sector_risk_contrib(df).show()

In [162]:
country_risk_contrib(df).show()

The majority of the risk comes unsurprisingly from its largest holdings.

### 2. Portfolio Return Analysis

To quantify realised performance, we compute **periodic portfolio returns** from asset price (or index) levels.  
Let $P_{i,t}$ be the price (or total-return index level) of asset $i$ at time $t$, and $w_{i,t-1}$ the weight held at the **start** of period $t$.

**Step 1 — Asset-level returns**  
Simple returns:  
$$
r_{i,t}=\frac{P_{i,t}}{P_{i,t-1}}-1
$$
Log returns (optional):  
$$
r_{i,t}^{(\log)}=\ln\!\left(\frac{P_{i,t}}{P_{i,t-1}}\right)
$$

**Step 2 — Portfolio-level return (pre-costs)**  
Using beginning-of-period weights:  
$$
R_{p,t}=\sum_{i=1}^{N} w_{i,t-1}\,r_{i,t}
$$

**Step 3 — Compounding over a horizon $T$**  
Cumulative simple return:  
$$
R_{p,1:T}=\prod_{t=1}^{T}\bigl(1+R_{p,t}\bigr)-1
$$
Cumulative log return (if using logs):  
$$
R_{p,1:T}^{(\log)}=\sum_{t=1}^{T} R_{p,t}^{(\log)}, \quad \text{where } R_{p,t}^{(\log)}=\ln\!\bigl(1+R_{p,t}\bigr)
$$

**Step 4 — Rebalancing through time**  
If the portfolio is rebalanced (e.g., monthly), compute $R_{p,t}$ each period using the updated weights $w_{i,t-1}$.  
Weights should satisfy $\sum_i w_{i,t-1}=1$ (or include cash if not fully invested).

---



In [163]:
plot_port_vs_bench(df)

In [164]:
fig, tri_df = portfolio_vs_benchmarks(df)
fig.show()

In [165]:
fig = monthly_bar_px_all(df)
fig.show()

The portfolio is clearly an actively managed fund with active weights against its benchmark. Over the period of 29 Sep 2022 to 29 Mar 2023, the fund has managed to outperform its benchmark as well as the MSCI world, the MSCI emerging market index and the JSE All Share index. Showing impressive skill from the fund manager.

### Retun Attribution

To gain a better understanding of where the outperformance has come from, we will perform a return attribution analysis on the portfolio. First we plot the esposures of the fund to sectors, countires and shares through time.

In [166]:
sector_exposure_area(df).show()
country_exposure_area(df).show()

The sector and country exposure are fairly stable through time. To gain better insight into how these have contributed to porfolio performance we look at the following:

In [167]:
sector_return_contrib_area(df).show()
country_return_contrib_area(df).show()

In [168]:
sector_return_contrib_totals_bar(df).show()
country_return_contrib_totals_bar(df).show()

It is clear that the information technology sector made the majority of the gains with a meaningful contribution from the communication services sector. China and Taiwan delivered the majority of the positive performance with Brazil the only large negative contributing region.

We can futher this analysis by looking into which share were the top 10 contributors and detractors to performance over the full period.

In [169]:
top_share_contributors_bar(df).show()
top_share_detractors_bar(df).show()

Taiwan Semiconductor manufact co was the notable performer over the period with a return of 10.5%. Tencent and Alibaba making meaningful contributions as well. On Hapvida and Meituan were the most negative dectractors to performance.

### 3. Risk Analysis

In this section we will explore the overall portfolio risk and how that has changed through time.

### Risk & Return Metrics — Definitions

Let daily portfolio and benchmark returns be $ r_{p,t} $ and $ r_{b,t} $ for $ t=1,\dots,T $.  
Define active return $ a_t = r_{p,t} - r_{b,t} $.  
Let $ D $ be trading days per year (use  $ D=252 $).

#### Annualised Return
$$
\left(\prod_{t=1}^{T} (1+r_{p,t})\right)^{\frac{D}{T}} - 1
$$


#### Volatility (Annualised)
$$
\sigma_p^{\text{ann}} = \sqrt{D}\;\mathrm{stdev}(r_{p,t})
$$

#### Tracking Error (Annualised)
$$
\mathrm{TE}^{\text{ann}} = \sqrt{D}\;\mathrm{stdev}(a_t)
$$

#### Sharpe Ratio (Annualised)
With daily risk-free rate \( r_{f,t} \) (often set to 0):
$$
\text{Sharpe} =
\frac{\sqrt{D}\;\overline{(r_{p,t}-r_{f,t})}}{\mathrm{stdev}(r_{p,t}-r_{f,t})}
\approx
\frac{\sqrt{D}\;\overline{r_{p,t}}}{\mathrm{stdev}(r_{p,t})}
$$

#### Information Ratio (Annualised)
$$
\text{IR} = \frac{\sqrt{D}\;\overline{a_t}}{\mathrm{stdev}(a_t)}
$$

#### Max Drawdown (Portfolio)
Let $ V_t = \prod_{i=1}^{t} (1+r_{p,i}) $ and $ P_t = \max_{1\le i\le t} V_i $.
$$
\text{MDD} = \min_{1\le t\le T} \left( \frac{V_t}{P_t} - 1 \right)
$$


In [170]:
risk_table = portfolio_risk_stats(df)
display(risk_table.style.format("{:.2%}"))

,Value
Ann. Return (Portfolio),49.95%
Ann. Return (Benchmark),19.85%
Active Return (Ann.),30.09%
Volatility (Ann.),17.43%
Tracking Error (Ann.),10.95%
Sharpe Ratio,232.58%
Information Ratio,204.91%
Max Drawdown,-7.38%


As we can see the stats are slightly unreliable due to the lack of data (less than a year) which effects the results. However, it is clear that the portfolio has returned impressive returns above the benchmark and has done so with an impressive information ratio and sharpe ratio. Showing alpha generated above that of the benchmark and that the risk taken has been adequetly compensated for.

In [171]:
drawdown_px(df).show()

In [172]:
beta_avg_series_px(df).show()
sector_beta_bar_px(df).show() 

What we can gain from these charts is that the portfolio has on average a higher beta than 1 meaning that it has a moderate pro-risk bias. Consumer discetionary and communication sectors are above 1 as expected given the cyclical nature of these sectors.

In [173]:
sector_risk_contrib_area(df, colors=ninetyone_colors).show()
country_risk_contrib_area(df, colors=ninetyone_colors).show()

In [174]:
sector_risk_act_contrib_area(df, colors=ninetyone_colors).show()
country_risk_act_contrib_area(df, colors=ninetyone_colors).show()

Both the country and sector contribution to active and total risk stay fairly consitent through time, roughly in line with the broad exposures of the fund itself.

### 4. ESG Analysis

In this section we explore the overall ESG rating of the fund through time as well as finding the source of the score.

In [175]:
fig_portfolio_esg(df).show()
fig_portfolio_esg_pillars(df).show()
fig_sector_esg(df).show()

In [176]:
fig_portfolio_esg_pillars_sector(df, sector="Materials").show()

The portfolio is ESG cognisant with a focus on the environmental pillar of ESG. The overall ESG score decreased slightly over the period due to a slight decrease in weighted governance score. While the environmental pillar continued to improve over the period. The weighted average of each sector to overall ESG score is shown in the following chart showing how each sector has contirbuted on a relative basis to the overal ESG score of the portfolio. 

As a scanity check we plot the equally weighted ESG score for the Materials sector where we see, as expected, its environmental score is low. This further confirms that the portfolio is exposed minimally to the Materials sector.

### Final Remarks

This is an actively managed equity portfolio with a global, and specifically emerging market, focus. It is clearly ESG-conscious, with particular emphasis on the environmental pillar of ESG.

The portfolio manager has successfully generated alpha relative to the benchmark and has also outperformed global equity indices that may be relevant for peer comparison. Its concentration in the Information Technology sector, particularly in Taiwan, has driven the majority of returns over the period.